In [1]:
from dynesty import NestedSampler
from dynesty import plotting as dyplot
from dynesty import utils as dyfunc


import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

from plotbin.plot_velfield import plot_velfield
from jampy.mge_radial_mass import mge_radial_mass
from jampy.mge_half_light_isophote import mge_half_light_isophote
from astropy import units as u

In [2]:
result_path     = "./313415/model1/JAM/"  #path to the non-linear results
data_path = "/home/carlosmelo/Documents/GitHub/Illustris/my_illustris/TNG50-1-snap84-subhalo313415" #path to the data
analysis_path = result_path+"Analysis/" #path where the analysis will be saved

os.makedirs(analysis_path) 

with open(result_path+f'/Final_sampler.pickle','rb') as f:
    sampler = pickle.load(f)
    f.close()

with open(result_path+f'/JAM_class.pickle','rb') as f:
    Jam_Model = pickle.load(f)
    f.close()
    
with open(result_path+f'/priors.pickle','rb') as f:
    priors = pickle.load(f)
    f.close()

def resume_dlogz(sampler):
    results = sampler.results
    logz_remain = np.max(sampler.live_logl) + results.logvol[-1]
    delta_logz = np.logaddexp(results.logz[-1], logz_remain) - results.logz[-1]

    return delta_logz


In [3]:
sampler.results.summary()

Summary
nlive: 500
niter: 5404
ncall: 74312
eff(%):  7.945
logz: -13.036 +/-  0.151


In [4]:
# Generate a new set of results with statistical+sampling uncertainties.
labels = list(priors.keys())
parsRes = priors.copy()
results_sim = dyfunc.jitter_run(dyfunc.resample_run(sampler.results))
samples_sim, weights = results_sim.samples, results_sim.importance_weights()
quantiles = [dyfunc.quantile(samps, [0.16, 0.5,  0.84], weights=weights)
            for samps in samples_sim.T]                        #quantiles for each parameter

    #Update the parameters
for i, key in enumerate(parsRes.keys()):
    parsRes[key] = quantiles[i][1]
    print(key, parsRes[key])


inc 82.58677007736902
beta 0.00739519129570885
log_mbh 8.457953298614585
ml 3.2802575107580925
log_M0 12.398349041652494
rc 5.418295514995624
rs 11.761142961692574
slope 1.1814161993810297


In [5]:
fig, axes = dyplot.traceplot(results=results_sim, show_titles=True,
                             labels=labels,
                             )
fig.tight_layout()
plt.savefig(analysis_path+"/tracer_plot.png")
plt.close()

In [6]:
# Plot the 2-D marginalized posteriors.
cfig, caxes, = dyplot.cornerplot(results_sim, smooth=0.08,
                                show_titles=True,labels=labels,
                         )
fig.tight_layout()
plt.savefig(analysis_path+"/corner_plot.png")
plt.close()

In [7]:
Jam_Model.Updt_Model(UpdtparsDic=parsRes)
plt.figure(figsize=(18,10))
rmsModel, ml, chi2, flux  = Jam_Model.run_current(quiet=False, 
                                                  plot=True,nodots=False, 
                                                  cmap="rainbow", label=r"km/s")
plt.tight_layout()
plt.savefig(analysis_path+"/model.png")
plt.close()

Parameters Updated!
jam_axi_rms elapsed time sec: 0.09
No PSF convolution: sigmapsf == 0;
inc=82.6 beta_z=0.01 M/L=1.00 BH=2.87e+08 chi2/DOF=0.171
Total mass MGE: 1.856e+14


In [8]:
fig = plt.figure(figsize=(18, 10))
plot_velfield(Jam_Model.xbin, Jam_Model.ybin, 
            100*abs(Jam_Model.rms-rmsModel)/Jam_Model.rms,
            colorbar=True,  cmap="rainbow",  
            markersize=0.2, label="[%]")
plt.title(r"${\Delta V_{\rm rms}^{*}}$")

fig.tight_layout()
plt.savefig(analysis_path+"/residual.png")
plt.close()

In [9]:
import json
 # the json file where the output must be stored
out_descripition = open("{}/description.json".format(analysis_path), "w")
description = { "Reff":  "MGE effetive radius, in arcsec.",
                "R":     "Radius where quantities were measured, in arcsec.",
                "Mstar": "True stellar mass within R, in 1e10Msun.", 
                "Mdm":   "True dm mass within R, in 1e10Msun.", 
                "Mbh":   "True BH mass within R, in 1e10Msun.", 
                "Mtotal":"True total mass within R, in 1e10Msun.", 
                "fdm":   "Fraction of DM within R.",
                "MMstar": "Model stellar mass within R, in 1e10Msun.", 
                "MMdm":   "Model dm mass within R, in 1e10Msun.", 
                "MMbh":   "Model BH mass within R, in 1e10Msun.", 
                "MMtotal":"Model total mass within R, in 1e10Msun.", 
                "Mfdm":   "Model DM fraction within R.",
                "Dstar":  "(MMstar - Mstar)/Mstar",
                "Ddm":    "(MMdm - Mdm)/Mdm",
                "Dtotal": "(MMtotal - Mtotal)/Mtotal",
                "Dfdm":   "(Mfdm - fdm)/fdm"
               }
json.dump(description, out_descripition, indent = 8)
out_descripition.close()

In [10]:
# Get the effective radius in arcsec, and other quantities
# See mge_half_light_isophote documentation for more details
reff, reff_maj, eps_e, lum_tot = mge_half_light_isophote(Jam_Model.surf_lum,
                                             Jam_Model.sigma_lum,
                                             Jam_Model.qobs_lum,
                                             Jam_Model.distance)

In [11]:
#  Model
R     = 2.5*reff    # 2.5 Reff in arcsec
R_kpc = ( (R*u.arcsec * Jam_Model.distance*u.Mpc ).to(
                    u.kpc,u.dimensionless_angles()) ).value # 2.5Reff in kpc


    # Get the radial mass of stars and DM within rad
MMstar = float ( mge_radial_mass(Jam_Model.surf_lum * Jam_Model.ml_model, 
                               Jam_Model.sigma_lum, Jam_Model.qobs_lum,
                               Jam_Model.inc, R, Jam_Model.distance) )

MMdm = float ( mge_radial_mass(Jam_Model.surf_dm, 
                               Jam_Model.sigma_dm, Jam_Model.qobs_dm,
                               Jam_Model.inc, R, Jam_Model.distance) )
MMbh = float ( Jam_Model.mbh )  # Model BH mass

    # Total Mass
MMtotal = MMstar + MMdm + MMbh
    # Dark matter fraction
Mfdm = MMdm / MMtotal

In [12]:
# Data
    # Changes this path to where are your data


    # Load the snapshot data
dm_dataset   = np.load(data_path+"/dm/coordinates_dark.npy")
star_dataset = np.load(data_path+"/imgs/coordinates_star.npy")
    
    # Stellar content
rStar = np.sqrt(np.sum(star_dataset[:, 0:3]**2, axis=1)) # radius
i = rStar <= R_kpc                                     # only particles within 2.5Reff
Mstar = sum(star_dataset[:, 6][i]*1e10)                  # stellar mass within 2.5Reff



    # Dark content
rDM = np.sqrt(np.sum(dm_dataset[:, 0:3]**2, axis=1))
i = rDM <= R_kpc
Mdm = sum(dm_dataset[:,3][i]*1e10)





Mbh = (0.015411)*(1e10/0.67) # BH mass from TNG catalogue
    # Total snapshot mass within 2.5Reff
Mtotal = Mstar + Mdm + Mbh
    # DM fraction in the snapshot within 2.5Reff
fdm = Mdm/Mtotal

FileNotFoundError: [Errno 2] No such file or directory: '/home/carlosmelo/Documents/GitHub/Illustris/my_illustris/TNG50-1-snap84-subhalo313415/dm/coordinates_dark.npy'

In [ ]:
print("Model stellar Mass: {:.2e} Msun".format( float(MMstar) ))
print("Data  stellar Mass: {:.2e} Msun".format( float(Mstar) ))
Dstar = float ( (MMstar - Mstar)/Mstar )
print("(Model - Data)/Data: {:.2f}".format( float(Dstar) ))

In [13]:
print("Model dm Mass: {:.2e} Msun".format( float(MMdm) ))
print("Data  dm Mass: {:.2e} Msun".format( float(Mdm) ))
Ddm = float ( (MMdm - Mdm)/Mdm )
print("(Model - Data)/Data: {:.2f}".format( float(Ddm) ))

Model dm Mass: 9.25e+09 Msun


NameError: name 'Mdm' is not defined

In [34]:
print("Model total Mass: {:.2e} Msun".format( float(MMtotal) ))
print("Data  total Mass: {:.2e} Msun".format( float(Mtotal) ))
Dtotal = float ( (MMtotal - Mtotal)/Mtotal )
print("(Model - Data)/Data: {:.2f}".format( float(Dtotal) ))

Model total Mass: 3.41e+10 Msun
Data  total Mass: 4.12e+10 Msun
(Model - Data)/Data: -0.17


In [35]:
print("Model DM fraction: {:.2f}".format( float(Mfdm) ))
print("Data  DM fraction: {:.2f}".format( float(fdm) ))
Dfdm = float ( (Mfdm - fdm)/fdm )
print("(Model - Data)/Data: {:.2f}".format( float(Dfdm) ))

Model DM fraction: 0.17
Data  DM fraction: 0.48
(Model - Data)/Data: -0.64


In [22]:
# Radial Density profile
from jampy.mge_radial_density import mge_radial_density

radii    = np.arange(0.1, 10*reff, 0.01)   # Radii in arcsec
pc       = Jam_Model.distance*np.pi/0.648  # Constant factor to convert arcsec --> pc
radii_pc = radii*pc                        # Radii in pc

dstar = mge_radial_density(Jam_Model.surf_lum * Jam_Model.ml_model, 
                               Jam_Model.sigma_lum, Jam_Model.qobs_lum,
                               Jam_Model.inc, radii, Jam_Model.distance)

ddm = mge_radial_density(Jam_Model.surf_dm, 
                               Jam_Model.sigma_dm, Jam_Model.qobs_dm,
                               Jam_Model.inc, radii, Jam_Model.distance)

# Load DM density profile
from astropy.io import fits
dm_hdu = fits.open(data_path+"/dm/density_fit.fits")
true_density = dm_hdu[1].data["density"]
true_radii = dm_hdu[1].data["radius"]
dm_fit = dm_hdu[1].data["bestfit"]

i = true_radii < radii_pc.max()

true_density = true_density[i]
true_radii   = true_radii[i]
dm_fit = dm_fit[i]

#  Load star density profile
star_hdu = fits.open(data_path+"/imgs/stellar_density.fits")
rho_stars = star_hdu[1].data["density"]
r_star = star_hdu[1].data["radius"]


i = r_star < radii_pc.max()

rho_stars = rho_stars[i]
r_star   = r_star[i]


In [24]:
import matplotlib.pyplot as plt
plt.rcParams['xtick.labelsize']= 12
plt.rcParams['ytick.labelsize']= 12


plt.figure(figsize=(15,8))

plt.plot(radii_pc, np.log10(dstar), label="Star", color="red")
plt.plot(radii_pc, np.log10(ddm), label="DM", color="magenta")
plt.plot(radii_pc, np.log10(ddm+dstar), label="Total", color="black")
plt.plot(true_radii, np.log10(dm_fit), label="DM CgNFW Fit", color="blue", markersize=12)
#plt.plot(radii_pc, np.log10(a), label="MGE", color="black")

plt.plot(true_radii, np.log10(true_density),  ".", label="DM data", color="magenta", markersize=12)

plt.plot(r_star, np.log10(rho_stars),  ".", label="Stars data", color="red", markersize=12)
plt.plot(r_star, np.log10(rho_stars+true_density),  ".", label="Total data", color="black", markersize=12)




plt.xlabel("radii [pc]", size=20)
plt.ylabel("$\log_{10}(\\frac{\\rho}{M_\odot/pc^3})$", size=20)
plt.legend()
plt.xscale("log")
plt.tight_layout()
plt.savefig(analysis_path+"/density_profiles.png")
plt.close()

In [39]:
r = {}
r["Reff"]   = reff
r["R"]      = R
r["Mstar"]  = Mstar
r["Mdm"]    = Mdm
r["Mbh"]    = Mbh
r["Mtotal"] = Mtotal
r["fdm"]    = fdm
r["MMstar"]  = MMstar
r["MMdm"]    = MMdm
r["MMbh"]    = MMbh
r["MMtotal"] = MMtotal
r["Mfdm"]    = Mfdm
r["Dstar"]   = Dstar
r["Ddm"]     = Ddm
r["Dtotal"]  = Dtotal
r["Dfdm"]    = Dfdm
# the json file where the output must be stored
out_r = open("{}/quantities.json".format(analysis_path), "w")
json.dump(r, out_r, indent = 8)
out_r.close()



----